<a href="https://colab.research.google.com/github/hsieh2000/HW/blob/main/%E5%84%AA%E5%8C%96%E7%A9%BA%E9%96%93.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install fake_useragent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 5.0 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


新境外基金


In [5]:
import time
import datetime
import json 
import math
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
import random
import warnings
import os


# 建立資料夾
newest_path = './最新檔/'
log_path = './境外基金基本資訊記錄檔/'
dir = ["csv/", "txt/", "err/"]

for i in dir:
  if not os.path.isdir(log_path+i):
    os.makedirs(log_path+i)

if not os.path.isdir(newest_path):
    os.mkdir(newest_path)

# 路徑設定
config_path = "/content/drive/MyDrive/config.json"
filename = "境外基金基本資訊"
bug_url = "境外基金錯誤連結"
csv_path = log_path+dir[0]
txt_path = log_path+dir[1]
err_path = log_path+dir[2]
# bug_fund_path = "./"
# output_path = './'


# 參數設定
delay = random.uniform(1, 5)
warnings.filterwarnings("ignore")
user_agent = UserAgent()
now = datetime.datetime.now().strftime("%Y%m%d")    
header = {"user-agent" : user_agent.random,
      "Referer" : "https://announce.fundclear.com.tw/MOPSFundWeb/E03_01.jsp"}
link_header = 'https://announce.fundclear.com.tw/MOPSFundWeb/'


class newForeignFund(object):

    def __init__(self):
        with open(config_path) as f:
            config = json.load(f)
            target_agent_list = config['agent']
            print(target_agent_list)
            with open(f'{txt_path}{filename}{now}.txt', mode='w') as T:
                T.write('{\'funds\':[')

        a = self.general_agent()
        a = self.compare(a,target_agent_list)
        a = self.tab_link_generaltor(a)
        a = self.meta_link_generator(a)
        a = self.meta_crawler(a)
        a = self.toDataframe(a)
        a = self.store(a)

    def general_agent(self):
        url = 'https://announce.fundclear.com.tw/MOPSFundWeb/O01.jsp?organizType=ALL&organizCom=ALL&fundCom=ALL'
        header = {"user-agent" : user_agent.random}
        res=requests.get(url,headers = header, timeout=30)
        soup = BeautifulSoup(res.text, 'html.parser')
        webText = str(soup.select("select[name='organizCom'] option"))
        regex = re.compile('\w\d{4}">\w+')
        result = re.findall(regex, webText)   
        result = result[2:]    
        agent_name = list(map(lambda x : x.split('">')[1],result))
        agent_ID = list(map(lambda x : x.split('">')[0],result))
        print(agent_name)
        return agent_ID

    def compare(self, agent_ID, target_list):
        # target_list = ['A0038','A0031','A0018','B0029','A0036','A0011','A0032']
        target_Id = list(set(target_list).intersection(set(agent_ID)))
        print(target_Id)
        return target_Id

    def tab_link_generaltor(self, target_Id):
        regex2 = re.compile("A01_01.jsp?.+establishYear=ALL")
        link_header = 'https://announce.fundclear.com.tw/MOPSFundWeb/'
        target_link_list=[]

        for ID in target_Id:
            agent_url = f'https://announce.fundclear.com.tw/MOPSFundWeb/O01.jsp?organizType=ALL&organizCom={ID}&fundCom=ALL'
            res=requests.get(agent_url,headers = header, timeout=10)
            soup = BeautifulSoup(res.text, 'html.parser')
            AgentWebText = soup.select('td[style="cursor: hand; color: blue"]')
            link = [ regex2.search(str(s)).group(0) for s in AgentWebText if regex2.search(str(s)) ]
            link = list(map(lambda x : link_header+x.replace(';','&').replace('®','&reg').replace('establishYear=ALL','establishYear=&pid='),link))
            target_link_list = target_link_list + link
            time.sleep(delay)
        return target_link_list

    def retry_page(self, url):
        times = 0
        while times<3:
            try:
                res=requests.get(url,headers = header)
                soup = BeautifulSoup(res.text, 'html.parser')
                return soup
            except:
                times+=1
                print(f"try {times} time, {url}")
                pass

    def ID_crawler(self, url, ID_link):
        main = 'main1.jsp?fundId='
        
        soup = self.retry_page(url)

        fundID = soup.select('tr.row1 td[style="cursor: hand;"]')+ soup.select('tr.row2 td[style="cursor: hand;"]')
        regex4 = re.compile("A01_02.jsp\?fundId=.+',")
        link2 = [regex4.search(str(s)).group(0) for s in fundID if regex4.search(str(s))]
        _ID_link = list(map(lambda x : link_header+main+x.replace("',","").split('=')[1],link2))
        ID_link = ID_link+_ID_link
        
        TdRight = soup.select('td[align="right"]')
        regex3 = re.compile("(.*)")    
        summary = regex3.search(str(TdRight)).group(0)
        try:
            full_page = int(re.search('\d{1,2}',summary.split(',')[0]).group(0))
        except:
            full_page = 1

        time.sleep(delay)
        return (url, full_page, ID_link)

    def changing_page(self, firsturl, ID_link):
        i=1
        url = firsturl+str(i)
        print(url)
        temp = self.ID_crawler(url,ID_link)
        ID_link = temp[2]
        end_page = temp[1]
        url = temp[0]
        print(f'page1/{end_page}')

        for j in range(2,int(end_page)+1):
            url = url.split('pid=',1)[0]+'pid='+str(j)
            print(url)

            sub_temp = self.ID_crawler(url,ID_link)
            ID_link=sub_temp[2]
            print(f'page{j}/{end_page}')
            time.sleep(delay)
        return ID_link

    def meta_link_generator(self, target_link_list):
        a = []
        ID_link = []
        for x,y in enumerate(target_link_list):
            a = a + self.changing_page(y, ID_link)
            print(f'{x+1}/{len(target_link_list)}')
        return a

    def Replace(self, x):
        x = x.text
        x = x.replace('\t','').replace('\n','').replace('\xa0','').replace('\r','').replace('\"','')
        return x

    def ToJSON(self, keys, values):
        a = list(map(lambda x : self.Replace(x), keys))
        b = list(map(lambda x : self.Replace(x), values))
        res = {a[i]: b[i] for i in range(len(keys))}
        json_object = json.loads(json.dumps(res, ensure_ascii=False))#json.dumps回傳type為str，要用;json.loads()轉為dict
        return json_object

    def Convert(self, fund_name):
        name = '基金名稱'
        res_dct = {name : fund_name}
        res_dct = json.loads(json.dumps(res_dct, ensure_ascii=False))#json.dumps回傳type為str，要用;json.loads()轉為dict
        return res_dct

    def get_source(self, url):
        times = 0
        while times<3 :
            try:
                res = requests.get(url, headers=header)
                soup = BeautifulSoup(res.text, 'html.parser')
                c_name = soup.select('font[color="#003399"]')[0]
                return (soup, c_name)
            except:
                times+=1
                print(f"try {times} time, {url}")
                pass


    def meta_crawler(self, link_list):
        with open(f'{err_path}{bug_url}.txt', mode='w') as D:
            D.write("Error url:\n")
            D.close()
        json_list=[]
        for num, i in enumerate(link_list):
            try:
                soup, c_name = self.get_source(i)
                c = self.Convert(self.Replace(c_name))
                title = soup.select('.FieldTitle')
                content = soup.select('.FieldContent')
                j = self.ToJSON(title, content)
                c.update(j)
                with open(f'{txt_path}{filename}{now}.txt', mode='a') as T:
                    T.write(str(c))
                    if num+1 == len(link_list):
                      pass
                    else:
                      T.write(',')
                      
                json_list.append(c)
                time.sleep(delay)
            except:
                with open(f'{err_path}{bug_url}.txt', mode='a') as D:
                            D.write(f'{i},')
                            D.write("\n")
                            D.close() 
                print(f"{i} got problem")
                pass      
            #測試
            if num ==10:
              break

            print(f'{self.Replace(c_name)}, {num+1}/{len(link_list)}')
        with open(f'{txt_path}{filename}{now}.txt', mode='a') as T:
            T.write(']}')
            T.close()       
        return json_list

    def toDataframe(self, json_list):
        df = pd.DataFrame(json_list,columns=list(json_list[0].keys()))
        return df

    def store(self, df):
        df.to_csv(f'{csv_path}{filename}{now}.csv', encoding='utf-8')
        df.to_csv(f'{newest_path}{filename}.csv', encoding='utf-8')

    # target_list = ['A0038','A0031','A0018','B0029','A0036','A0011','A0032']

if __name__ == '__main__':
    c = newForeignFund()



['B0029', 'A0018', 'A0011', 'A0038', 'A0036', 'A0031', 'A0032']
['景順證券投資信託股份有限公司', '瀚亞證券投資信託股份有限公司', '保德信證券投資信託股份有限公司', '摩根證券投資信託股份有限公司', '新光證券投資信託股份有限公司', '瑞銀證券投資信託股份有限公司', '台中銀證券投資信託股份有限公司', '聯博證券投資信託股份有限公司', '柏瑞證券投資信託股份有限公司', '中國信託證券投資信託股份有限公司', '宏利證券投資信託股份有限公司', '貝萊德證券投資信託股份有限公司', '野村證券投資信託股份有限公司', '鋒裕匯理證券投資信託股份有限公司', '安聯證券投資信託股份有限公司', '富達證券投資信託股份有限公司', '德銀遠東證券投資信託股份有限公司', '凱基證券投資信託股份有限公司', '施羅德證券投資信託股份有限公司', '合作金庫證券投資信託股份有限公司', '大華銀證券投資信託股份有限公司', '路博邁證券投資信託股份有限公司', '康和證券投資顧問股份有限公司', '富蘭克林證券投資顧問股份有限公司', '萬寶證券投資顧問股份有限公司', '永豐證券投資顧問股份有限公司', '法銀巴黎證券投資顧問股份有限公司', '安睿宏觀證券投資顧問股份有限公司', '霸菱證券投資顧問股份有限公司', '全球證券投資顧問股份有限公司', '中租證券投資顧問股份有限公司', '國泰證券投資顧問股份有限公司', '富盛證券投資顧問股份有限公司', '百達證券投資顧問股份有限公司', '品浩太平洋證券投資顧問股份有限公司', '瑞聯證券投資顧問股份有限公司']
['A0031', 'A0036', 'A0038', 'A0018', 'B0029', 'A0032', 'A0011']
https://announce.fundclear.com.tw/MOPSFundWeb/A01_01.jsp?agent=A0031&amp&fundCom=042&amp&fundType=ALL&amp&fundInvArea=&amp&fundInvType=&amp&fundAsset=&amp&fundAssetD=&registerArea=ALL&amp&fundShare=AL

In [ ]:
# Download the file.
from google.colab import files
files.download('境外基金基本資訊.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

公開消息

In [ ]:
import datetime
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
import time
import json 
import math
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
import random
import warnings
import os

# 建立資料夾
newest_path = './最新檔/'
log_path = './公開消息記錄檔/'
dir = ["csv/", "txt/", "err/"]

for i in dir:
  if not os.path.isdir(log_path+i):
    os.makedirs(log_path+i)

if not os.path.isdir(newest_path):
    os.mkdir(newest_path)

# 路徑設定
config_path = '/content/drive/MyDrive/config.json'
bug_url = "公告消息錯誤連結"
csv_path = log_path+dir[0]
txt_path = log_path+dir[1]
err_path = log_path+dir[2]

# 參數設定
delay = random.uniform(1, 5)
warnings.filterwarnings("ignore")
user_agent = UserAgent()
# now = datetime.datetime.now().strftime("%Y-%m-%d")
delay = random.uniform(1, 4)
header = {
    'Host' : 'announce.fundclear.com.tw',
    'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'user-agent': user_agent.random,
    'connectoin':'close'
}

with open(config_path) as f:
    config = json.load(f)
    agent_list = config['agent']
    startYear = config['startDate'].split("-")[0]
    startMonth = config['startDate'].split("-")[1]
    endYear = config['endDate'].split("-")[0]
    endMonth = config['endDate'].split("-")[1]

    date1 = f"{startYear}-{startMonth}"
    date2 = f"{endYear}-{endMonth}"
    if date1 == date2:
      filename = f"公告消息{startYear}{startMonth}"
    else:
      filename = f"公告消息{startYear}{startMonth}~{endYear}{endMonth}"

class bulletin(object):
    def __init__(self):
        with open(f"{txt_path}{filename}.txt", mode='w') as T:
            T.write('{\'data\':[')
        self.bulletinIDcrawler(agent_list, startYear, startMonth, endYear, endMonth)

    def bulletinIDcrawler(self, agent_list, startYear, startMonth, endYear, endMonth):
        link_list = []
        for agent in agent_list:
            url = f'https://announce.fundclear.com.tw/MOPSFundWeb/G01_01.jsp?fundHouseId=all&fundId=all&MsgLevelId=0&AgentId={agent}&baseBeginDate={startYear}&baseEndDate={startMonth}&baseBeginDate1={endYear}&baseEndDate1={endMonth}&qryText='
            res = requests.get(url, headers=header, timeout=30)
            print("loading...")
            soup = BeautifulSoup(res.text, 'html.parser')
            a = soup.select('h1 td[style="cursor: hand; color: royalblue"]')
            for j in a:
                link = re.search(r"\w+\.jsp\?seq=\d+",str(j))
                if bool(link):
                    link_list.append(link.group(0))
            print(f"{agent}")    
            time.sleep(delay)
        self.contentCrawler(link_list)    

    def Replace(self, x):
        x = x.replace('\n','').replace('\xa0','').replace('\r','').replace('\t','')
        return x

    def ToJSON(self, a,b):
        a = list(map(lambda x : self.Replace(x), a))
        b = list(map(lambda x : self.Replace(x), b))
        res = {a[i]: b[i] for i in range(len(a))}
        # c = list(map(lambda x, y: x + "':'" + y, a, b))
        # d = str(c).replace("[","{").replace("]","}").replace('\':\'','":"')
        json_object = json.loads(json.dumps(res))
        return json_object


    def get_source(self, url):
        times = 0
        while times<3 :
            try:
                res = requests.get(url, headers=header, timeout=30)
                soup = BeautifulSoup(res.text, 'html.parser')
                title = list(soup.select('table[cellpadding="3"] td[class="row2"]'))
                info = list(soup.select('table[cellpadding="3"] td[class="row1"]'))
                return (title, info)
            except:
                times+=1
                print(f"try {times} time, {url}")
                pass

    def contentCrawler(self, link_list):
        with open(f'{err_path}{bug_url}.txt', mode='w') as D:
            D.write("Error url:\n")
            D.close()
        url_head = 'https://announce.fundclear.com.tw/MOPSFundWeb/'
        bulletin_list = []
        for k,_ in enumerate(link_list):
            actual = url_head+_
            url = actual
            try:
                Get_source = self.get_source(url)
                title = Get_source[0]
                info = Get_source[1]

                # title = list(soup.select('table[cellpadding="3"] td[class="row2"]'))
                # info = list(soup.select('table[cellpadding="3"] td[class="row1"]'))

                title = list(map(lambda x: x.text.replace('：','') , title))
                info = list(map(lambda x: x.text.replace('\r','').replace('\n','').replace('\t','').replace('  ','') , info))

                temp = self.ToJSON(title, info)

                with open(f"{txt_path}{filename}.txt", mode='a') as T:
                    T.write(str(temp))
                    if k+1 == len(link_list):
                      pass
                    else:
                      T.write(',')
                bulletin_list.append(temp)
                print(f"{k}/{len(link_list)}")
                time.sleep(delay)
            except:
                with open(f'{err_path}{bug_url}.txt', mode='a') as D:
                            D.write(f'{url},')
                            D.write("\n")
                            D.close() 
                print(f"{url} got problem")
                pass  
            #測試 
            # if k == 10:
            #     break    

        with open(f"{txt_path}{filename}.txt", mode='a') as T:
            T.write(']}')        
        self.store(bulletin_list, temp)

    def store(self, bulletin_list, temp):
      df = pd.DataFrame(columns = list(temp.keys()), data = bulletin_list)
      df.to_csv(f"{csv_path}{filename}.csv", encoding='utf-8')
      df.to_csv(f"{newest_path}{filename}.csv", encoding='utf-8')
if __name__ == "__main__":
    c = bulletin()
    

loading...
B0029
0/80
1/80
2/80
3/80
4/80
5/80
6/80
7/80
8/80
9/80
10/80
11/80
12/80
13/80
14/80
15/80
16/80
17/80
18/80
19/80
20/80
21/80
22/80
23/80
24/80
25/80
26/80
27/80
28/80
29/80
30/80
31/80
32/80
33/80
34/80
35/80
36/80
37/80
38/80
39/80
40/80
41/80
42/80
43/80
44/80
45/80
46/80
47/80
48/80
49/80
50/80
51/80
52/80
53/80
54/80
55/80
56/80
57/80
58/80
59/80
60/80
61/80
62/80
63/80
64/80
65/80
66/80
67/80
68/80
69/80
70/80
71/80
72/80
73/80
74/80
75/80
76/80
77/80
78/80
79/80


國內投資人持有金額

In [ ]:
import datetime
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
import time
import json 
import math
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
import random
import os
import warnings

# 建立資料夾
newest_path = './最新檔/'
log_path = './國內投資人持有金額記錄檔/'
dir = ["csv/", "txt/", "err/"]

for i in dir:
  if not os.path.isdir(log_path+i):
    os.makedirs(log_path+i)

if not os.path.isdir(newest_path):
    os.mkdir(newest_path)

# 路徑設定
config_path = '/content/drive/MyDrive/config.json'
csv_path = log_path+dir[0]
txt_path = log_path+dir[1]
err_path = log_path+dir[2]

# 參數設定
delay = random.uniform(1, 5)
warnings.filterwarnings("ignore")
useragent = UserAgent()
header = {
    'Host' : 'announce.fundclear.com.tw',
    'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    "Referer": "https://announce.fundclear.com.tw",   
    "user-agent" : useragent.random
}

with open(config_path) as f:
    config = json.load(f)
    now = datetime.datetime.now()
    now.strftime("%Y-%m")    

class domesticFund(object):
    def __init__(self):
        a = self.inputCheck()
        a = self.reasonableCheck()
        a = self.crawler(a)
        a = self.toDataframe(a[0],a[1])
        a = self.store(a[0],a[1])

    def inputCheck(self):
        check_start = re.search(r'\d{4}-\d{2}',config['startDate']).group(0)
        check_end = re.search(r'\d{4}-\d{2}',config['endDate']).group(0)
        print(check_start, check_end)

        if check_start and check_end:
            print(check_start,check_end)
        elif config['startDate']=="" and config['endDate'] =="latest":
            # self.newest()
            print('True')
        else:
            print('you should input year-month')
        return (check_start, check_end)

    def reasonableCheck(self):
        startDate = config['startDate'].split('-')
        endDate = config['endDate'].split('-')
        
        if int(endDate[0])-int(startDate[0])>1:
            year_gap = int(endDate[0])-int(startDate[0])-1
            year_gap = year_gap*12
            a = int(startDate[1])
            a = 12-a+1
            b = int(endDate[1])
            duration = a+year_gap+b 
            lst = self.MonthList(startDate, endDate, duration) 
        elif int(endDate[0])-int(startDate[0])==1:
            a = int(startDate[1])
            a = 12-a+1
            b = int(endDate[1])
            duration = a+b
            lst = self.MonthList(startDate, endDate, duration) 
        elif int(endDate[0])-int(startDate[0])==0:
            if int(startDate[1])-int(endDate[1]) <= 0:
                a = int(startDate[1])
                b = int(endDate[1])
                duration = b-a+1
                lst = self.MonthList(startDate, endDate, duration) 
            else:
              print("wrong period")
              duration = None
        else:
            print("Wrong period input")
            duration = None
        return lst

    def MonthList(self, startDate, endDate, duration):
        monthList = []
        t = date(int(startDate[0]),int(startDate[1]),1)
        for _ in range(duration):
            iterMonth = t.strftime("%m")
            iterYear = t.strftime("%Y")
            if int(iterYear)>int(date.today().strftime("%Y")):
                print('exceed this year')
                t = t+relativedelta(months=1)
            elif int(iterYear)==int(date.today().strftime("%Y")) and int(iterMonth)>int(date.today().strftime("%m")):
                print('exceed this month')
                t = t+relativedelta(months=1)
            else:
                r = t.strftime("%Y%m")
                print(r)
                monthList.append(r)
                t = t+relativedelta(months=1)
        print(monthList)
        return(monthList)

    def crawler(self, lst):
        x = '依受益人形態'
        y = '依計價幣別合計'
        data = []
        taggle = True
        for month in lst[::-1]:
            try:
                url = f"https://announce.fundclear.com.tw/MOPSFundWeb/N01.jsp?strYear={month}"
                session = requests.Session()
                res = session.get(url, headers = header)
                soup = BeautifulSoup(res.text, 'html.parser')
                a = soup.select('td')
                a = list(map(lambda x:str(x),a))
                raw = self.range_selector(a,x,y)
                if taggle ==True:
                    latest_month = month
                    print(latest_month)
                    if lst[0] == latest_month:
                        filename = f"國內投資人持有金額{latest_month}"
                    else:
                        filename = f"國內投資人持有金額{lst[0]}~{latest_month}"
                    with open(f'{txt_path}{filename}.txt', mode='w') as f:
                      f.write("{data:")
                    taggle = False
                print(url)
                amount = [i for i in raw if 'align="right"' in i]
                amount = list(map(lambda x : x.split('>')[1].split('<')[0].replace(' ','').replace(',',''), amount))
                word = [i for i in raw if 'align="right"' not in i]
                _order = re.compile('rowspan="\d{1,2}"')
                detail = [i.split('>')[1].split('<')[0] for i in word if not re.search(_order,i)]
                I = re.compile('依\w+')
                notI = re.compile('依\w+合計')
                Order = [i for i in word if re.search(I,i) and not re.search(notI,i)]
                OOrder = []
                for p in Order:
                    num = int(re.search('\d{1,2}',p).group(0))
                    for q in range(num+1):
                        OOrder.append(p.split('>')[1].split('<')[0])
                Json = list(map(lambda x,y,z:{
                      "分類":x,
                      "細項":y,
                      "國內投資人持有金額(單位：新台幣元)":z,
                      "年月":month
                },OOrder,detail,amount
                ))
                data = data+Json
            except:
              print(f"err: {url}")
        with open(f'{txt_path}{filename}.txt', mode='a') as f:
            f.write(f'{str(data)}}}')
        return (data,filename)

    def range_selector(self, lst, x, y):
        for i,j in enumerate(lst):
            if x in j:
                start = i
                break
        for i,j in enumerate(lst):
            if y in j:
                end = i+2
                break
        aws = lst[start:end]
        return aws

    def toDataframe(self, json_list,filename):
        df = pd.DataFrame(json_list,columns=list(json_list[0].keys()))
        return (df,filename)

    def store(self, df, filename):
        df.to_csv(f'{csv_path}{filename}.csv', encoding='utf-8')
        df.to_csv(f'{newest_path}{filename}.csv', encoding='utf-8')


if __name__ == "__main__":
    c = domesticFund()



2022-01 2022-11
2022-01 2022-11
202201
202202
202203
202204
202205
202206
202207
202208
202209
202210
202211
['202201', '202202', '202203', '202204', '202205', '202206', '202207', '202208', '202209', '202210', '202211']
err: https://announce.fundclear.com.tw/MOPSFundWeb/N01.jsp?strYear=202211
err: https://announce.fundclear.com.tw/MOPSFundWeb/N01.jsp?strYear=202210
202209
https://announce.fundclear.com.tw/MOPSFundWeb/N01.jsp?strYear=202209
https://announce.fundclear.com.tw/MOPSFundWeb/N01.jsp?strYear=202208
https://announce.fundclear.com.tw/MOPSFundWeb/N01.jsp?strYear=202207
https://announce.fundclear.com.tw/MOPSFundWeb/N01.jsp?strYear=202206
https://announce.fundclear.com.tw/MOPSFundWeb/N01.jsp?strYear=202205
https://announce.fundclear.com.tw/MOPSFundWeb/N01.jsp?strYear=202204
https://announce.fundclear.com.tw/MOPSFundWeb/N01.jsp?strYear=202203
https://announce.fundclear.com.tw/MOPSFundWeb/N01.jsp?strYear=202202
https://announce.fundclear.com.tw/MOPSFundWeb/N01.jsp?strYear=202201


In [ ]:
歷史淨值

In [15]:
import datetime
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
import time
import json 
import math
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
import os
import random
import warnings
delay = random.uniform(1, 5)
warnings.filterwarnings("ignore")
useragent = UserAgent()

# 建立資料夾
newest_path = './最新檔/'
log_path = './歷史淨值記錄檔/'
dir = ["csv/", "txt/", "err/"]

for i in dir:
  if not os.path.isdir(log_path+i):
    os.makedirs(log_path+i)

if not os.path.isdir(newest_path):
    os.mkdir(newest_path)

# 路徑設定
config_path = "/content/drive/MyDrive/config.json"
basicInfo_path = './最新檔/境外基金基本資訊.csv'
bug_fund = "歷史淨值錯誤基金代碼"
csv_path = log_path+dir[0]
txt_path = log_path+dir[1]
err_path = log_path+dir[2]
c = pd.read_csv(f'{basicInfo_path}')

header = {
    'Host' : 'announce.fundclear.com.tw',
    'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    "Referer": "https://announce.fundclear.com.tw",   
    "user-agent" : useragent.random
}

with open(f'{config_path}') as f:
    config = json.load(f)
    agent_list = config['agent']
    begin = config['startDate'].split('-')
    begin = list(map(lambda x:int(x), begin))
    end = config['endDate'].split('-')
    end = list(map(lambda x:int(x), end))

class Net(object):
    def __init__(self):
        a = self.date_list()
        a = self.crawler(a)
        a = self.store(a[0],a[1])

    def date_list(self):
        now = datetime.datetime.now().strftime("%Y/%m/%d")
        yesterday = datetime.datetime.now() - timedelta(days=1)
        yesterday = yesterday.strftime("%Y/%m/%d")
        beginDate = datetime.datetime(begin[0],begin[1], begin[2]).strftime("%Y/%m/%d")
        endDate = datetime.datetime(end[0],end[1], end[2]).strftime("%Y/%m/%d")
        date = datetime.datetime(begin[0],begin[1], begin[2]) - datetime.datetime(end[0],end[1], end[2])
        dates = []
        dt = datetime.datetime.strptime(beginDate, "%Y/%m/%d")
        date = dt.strftime("%Y/%m/%d")
        if beginDate < now and beginDate < endDate:
            if endDate < now:
                while date <= endDate:
                    dates.append(date)
                    dt = dt + datetime.timedelta(1)
                    date = dt.strftime("%Y/%m/%d")
            else:
                print(f"the end date precedes now,\nthe date range will be rearrange from {config['startDate']} to {yesterday}.")
                while date <= yesterday:
                    dates.append(date)
                    dt = dt + datetime.timedelta(1)
                    date = dt.strftime("%Y/%m/%d") 
        else:
            print('you can\'t start at today or future.')  
                      
        return dates

    def Retry(self, url, payload):
        times = 0
        while times<3 :
            try:
                session = requests.Session()
                res = session.post(url, headers = header, data = payload, timeout=30)
                soup = BeautifulSoup(res.text, 'html.parser')
                return soup
            except:
                times+=1
                print(f"try {times} time, {url}")
                pass

    def crawler(self, dates):
        with open(f'{err_path}{bug_fund}.txt', mode='w') as D:
            D.write("Error ID:\n")
            D.close()
        df = pd.DataFrame(columns = ["日期","基金代碼","淨值"])
        S = dates[0].split('/')
        E = dates[-1].split('/')
        filename = f"歷史淨值{S[0]}{S[1]}{S[2]}~{E[0]}{E[1]}{E[2]}"
        with open(f'{txt_path}{filename}.txt', mode='w') as f:
            f.write('{\"data\":[')

        for j,i in enumerate(c["基金代碼"]):
            try:
                payload = {
                    "agentId": "",
                    "fundId": i,
                    "days": "",
                    "beginDate": dates[0],
                    "endDate": dates[-1],
                    "checkSubmit": "Y"
                }
                with open(f'{txt_path}{filename}.txt', mode='a') as f:
                    f.write('{\"基金代碼\":')
                    f.write(f'\"{i}\",\"細項\":')

                url = 'https://announce.fundclear.com.tw/MOPSFundWeb/D02_02.jsp'

                soup = self.Retry(url, payload)

                raw = str(soup.select('script')[-1])
                date = re.compile("\d{4}/\d{2}/\d{2}")
                net = re.compile("\d+\.\d{6}")
                date = re.findall(date,raw)
                net = re.findall(net,raw)
                Jlist = list(map(lambda x,y : {
                    "日期":x,
                    "淨值":y
                },date,net ))
                with open(f'{txt_path}{filename}.txt', mode='a') as f:
                    f.write(str(Jlist).replace("\'","\""))
                    f.write("}")
                    if j+1 == len(c["基金代碼"]):
                      pass
                    else:
                      f.write(",")

                Jlist_df = list(map(lambda x,y : {
                    "日期":x,
                    "基金代碼":i,
                    "淨值":y
                },date,net ))
                print(Jlist_df[0])     
                print(type(Jlist_df[0]))
                # for k in Jlist_df:
                df = df.append(Jlist_df, ignore_index=True)
                print(f'{i},{j}/{len(c["基金代碼"])}')
                time.sleep(delay)
            except:
                with open(f'{err_path}{bug_fund}.txt', mode='a') as D:
                            D.write(f'{i},')
                            D.write("\n")
                            D.close() 
                print(f"{url} got problem")
                pass    
            #測試 
            if j == 10:
                break

        with open(f'{txt_path}{filename}.txt', mode='a') as f:
            f.write(']}')
        return (df,filename)

    def store(self, df, filename):
        print(f'export {filename}.csv')
        df.to_csv(f'{csv_path}{filename}.csv', encoding='utf-8')
        df.to_csv(f'{newest_path}{filename}.csv', encoding='utf-8')

        
if __name__ == "__main__":
    n = Net()
        






{'日期': '2022/11/02', '基金代碼': 'BLKAEMDU', '淨值': '35.830000'}
<class 'dict'>
BLKAEMDU,0/11
{'日期': '2022/11/02', '基金代碼': 'BLKAEUDE', '淨值': '164.330000'}
<class 'dict'>
BLKAEUDE,1/11
{'日期': '2022/11/02', '基金代碼': 'BLKAGMDU', '淨值': '30.440000'}
<class 'dict'>
BLKAGMDU,2/11
{'日期': '2022/11/02', '基金代碼': 'BLKANEDU', '淨值': '16.090000'}
<class 'dict'>
BLKANEDU,3/11
{'日期': '2022/11/02', '基金代碼': 'BLKAPAGT', '淨值': '16.370000'}
<class 'dict'>
BLKAPAGT,4/11
{'日期': '2022/11/02', '基金代碼': 'BLKAPEAU', '淨值': '10.750000'}
<class 'dict'>
BLKAPEAU,5/11
{'日期': '2022/11/02', '基金代碼': 'BLKBGGAT', '淨值': '17.980000'}
<class 'dict'>
BLKBGGAT,6/11
{'日期': '2022/11/02', '基金代碼': 'BLKBVAMA', '淨值': '107.890000'}
<class 'dict'>
BLKBVAMA,7/11
{'日期': '2022/11/02', '基金代碼': 'BLKBVPMA', '淨值': '106.510000'}
<class 'dict'>
BLKBVPMA,8/11
{'日期': '2022/11/02', '基金代碼': 'BLKCAAAH', '淨值': '12.430000'}
<class 'dict'>
BLKCAAAH,9/11
{'日期': '2022/11/02', '基金代碼': 'BLKCAPMA', '淨值': '119.080000'}
<class 'dict'>
BLKCAPMA,10/11
export 歷史淨值20221

每日匯率

In [ ]:
import datetime
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
import time
import json 
import math
import re
import pandas as pd
from pandas import json_normalize
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
import random
import warnings

# 建立資料夾
newest_path = './最新檔/'
log_path = './每日匯率記錄檔/'
dir = ["csv/", "txt/", "err/"]

for i in dir:
  if not os.path.isdir(log_path+i):
    os.makedirs(log_path+i)

if not os.path.isdir(newest_path):
    os.mkdir(newest_path)

# 路徑設定
config_path = "/content/drive/MyDrive/config.json"
csv_path = log_path+dir[0]
txt_path = log_path+dir[1]
err_path = log_path+dir[2]

# 參數設定
delay = random.uniform(1, 5)
warnings.filterwarnings("ignore")
useragent = UserAgent()
output_path = './'

header = {
    'Host' : 'announce.fundclear.com.tw',
    'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    "Referer": "https://announce.fundclear.com.tw",   
    "user-agent" : useragent.random
}
with open(config_path) as f:
    config = json.load(f)
    agent_list = config['agent']
    begin = config['startDate'].split('-')
    begin = list(map(lambda x:int(x), begin))
    end = config['endDate'].split('-')
    end = list(map(lambda x:int(x), end))

class exchange_rate(object):
    def __init__(self):
        a = self.date_list()
        a = self.crawler(a)
        a = self.store(a[0], a[1])

    def date_list(self):
        now = datetime.datetime.now().strftime("%Y/%m/%d")
        yesterday = datetime.datetime.now() - timedelta(days=1)
        yesterday = yesterday.strftime("%Y/%m/%d")
        beginDate = datetime.datetime(begin[0],begin[1], begin[2]).strftime("%Y/%m/%d")
        endDate = datetime.datetime(end[0],end[1], end[2]).strftime("%Y/%m/%d")
        date = datetime.datetime(begin[0],begin[1], begin[2]) - datetime.datetime(end[0],end[1], end[2])
        dates = []
        dt = datetime.datetime.strptime(beginDate, "%Y/%m/%d")
        date = dt.strftime("%Y/%m/%d")
        if beginDate < now and beginDate < endDate:
            if endDate < now:
                while date <= endDate:
                    dates.append(date)
                    dt = dt + datetime.timedelta(1)
                    date = dt.strftime("%Y/%m/%d")
            else:
                print(f"the end date precedes now,\nthe date range will be rearrange from {config['startDate']} to {yesterday}.")
                while date <= yesterday:
                    dates.append(date)
                    dt = dt + datetime.timedelta(1)
                    date = dt.strftime("%Y/%m/%d") 
        else:
            print('you can\'t start at today or in future.')  
                    
        return dates

    def retry(self, date):
        times = 0
        payload = {
            "navDate": date,
            "checkSubmit": "Y"
        }
        url = 'https://announce.fundclear.com.tw/MOPSFundWeb/R03.jsp'
        while times<3:
            try:
                session = requests.Session()
                res = session.post(url, headers = header, data = payload)
                soup = BeautifulSoup(res.text, 'html.parser')
                return soup
            except:
                times+=1
                print(f"try {times} time, {date}")
                pass

    def crawler(self, dates):
        if dates[0]==dates[-1]:
          filename = f"每日匯率{dates[0].replace('/','')}"
        else:
          filename = f"每日匯率{dates[0].replace('/','')}~{dates[-1].replace('/','')}"

        df = pd.DataFrame(columns = ["日期","幣別","買入匯率","賣出匯率"])
        with open(f"{txt_path}{filename}.txt", mode="w") as f:
            f.write("{\"data\":[")

        for num, D in enumerate(dates):

            soup = self.retry(D)
            try:
                table = soup.select('table[bgcolor="#FFFFFF"]')[1]
                table_head = table.select('[align="center"]')
                table_currency = table.select('[align="left"]')
                table_exchange = table.select('[align="right"]')

                table_head = list(map(lambda x : x.text,table_head))
                table_currency = list(map(lambda x : x.text,table_currency))
                table_exchange = list(map(lambda x : x.text,table_exchange))

                table_buy = []
                table_sale = []

                with open(f"{txt_path}{filename}.txt", mode="a") as f:
                    f.write("{\"date\":")
                    f.write(f'"{D}"')
                    f.write(",\"detail\":")

                for i, j in enumerate(table_exchange):
                    if i%2 == 0:
                        table_buy.append(j)
                    else:
                        table_sale.append(j)
                dictionary = list(map(lambda x,y,z : {
                    "幣別":x,
                    "買入匯率":y,
                    "賣出匯率":z
                },table_currency,table_buy,table_sale))
                Json = [json.dumps(i,ensure_ascii=False) for i in dictionary]
                with open(f"{txt_path}{filename}.txt", mode="a") as f:
                    f.write(str(Json).replace("\'",""))
                    f.write("}")
                    if num+1 == len(dates):
                      pass
                    else:
                      f.write(",")

                # data_for_df = {"date":D, "detail":Json}
                for i in dictionary:
                  df = df.append({"日期":D, "幣別":i["幣別"], "買入匯率":i["買入匯率"] , "賣出匯率":i["賣出匯率"]},ignore_index = True)
                print(f"{D} success")

            except:
                print(f"{D} fail")
                pass

        with open(f"{txt_path}{filename}.txt", mode="a") as f:
            f.write("]}")
        return (filename, df)

    def store(self, filename, df):
        df.to_csv(f"{csv_path}{filename}.csv", encoding="utf-8")
        df.to_csv(f"{newest_path}{filename}.csv", encoding="utf-8")
if __name__ == "__main__":
    c = exchange_rate()




2022/11/01 success
2022/11/02 success
2022/11/03 success
2022/11/04 success
2022/11/05 success


In [ ]:
import datetime
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
import time
import json 
import math
import re
import pandas as pd
from pandas import json_normalize
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
import random
import warnings
import os

config_path = "/content/drive/MyDrive/config.json"
with open(config_path) as f:
    config = json.load(f)
    root = config["root"]

# 建立資料夾
newest_path = f'{root}最新檔/'
log_path = f'{root}每日匯率記錄檔/'
dir = ["csv/", "txt/", "err/"]

for i in dir:
  if not os.path.isdir(log_path+i):
    os.makedirs(log_path+i)

if not os.path.isdir(newest_path):
    os.mkdir(newest_path)

# 路徑設定
csv_path = log_path+dir[0]
txt_path = log_path+dir[1]
err_path = log_path+dir[2]

# 參數設定
delay = random.uniform(1, 5)
warnings.filterwarnings("ignore")
useragent = UserAgent()

header = {
    'Host' : 'announce.fundclear.com.tw',
    'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    "Referer": "https://announce.fundclear.com.tw",   
    "user-agent" : useragent.random
}
with open(config_path) as f:
    config = json.load(f)
    agent_list = config['agent']
    begin = config['startDate'].split('-')
    begin = list(map(lambda x:int(x), begin))
    end = config['endDate'].split('-')
    end = list(map(lambda x:int(x), end))

class exchange_rate(object):
    def __init__(self):
        a = self.date_list()
        a = self.crawler(a)
        a = self.store(a[0], a[1])

    def date_list(self):
        now = datetime.datetime.now().strftime("%Y/%m/%d")
        yesterday = datetime.datetime.now() - timedelta(days=1)
        yesterday = yesterday.strftime("%Y/%m/%d")
        beginDate = datetime.datetime(begin[0],begin[1], begin[2]).strftime("%Y/%m/%d")
        endDate = datetime.datetime(end[0],end[1], end[2]).strftime("%Y/%m/%d")
        date = datetime.datetime(begin[0],begin[1], begin[2]) - datetime.datetime(end[0],end[1], end[2])
        dates = []
        dt = datetime.datetime.strptime(beginDate, "%Y/%m/%d")
        date = dt.strftime("%Y/%m/%d")
        if beginDate < now and beginDate < endDate:
            if endDate < now:
                while date <= endDate:
                    dates.append(date)
                    dt = dt + datetime.timedelta(1)
                    date = dt.strftime("%Y/%m/%d")
            else:
                print(f"the end date precedes now,\nthe date range will be rearrange from {config['startDate']} to {yesterday}.")
                while date <= yesterday:
                    dates.append(date)
                    dt = dt + datetime.timedelta(1)
                    date = dt.strftime("%Y/%m/%d") 
        else:
            print('you can\'t start at today or in future.')  
                    
        return dates

    def retry(self, date):
        times = 0
        payload = {
            "navDate": date,
            "checkSubmit": "Y"
        }
        url = 'https://announce.fundclear.com.tw/MOPSFundWeb/R03.jsp'
        while times<3:
            try:
                session = requests.Session()
                res = session.post(url, headers = header, data = payload)
                soup = BeautifulSoup(res.text, 'html.parser')
                return soup
            except:
                times+=1
                print(f"try {times} time, {date}")
                pass

    def crawler(self, dates):
        if dates[0]==dates[-1]:
          filename = f"每日匯率{dates[0].replace('/','')}"
        else:
          filename = f"每日匯率{dates[0].replace('/','')}~{dates[-1].replace('/','')}"

        df = pd.DataFrame(columns = ["date","detail"])
        with open(f"{txt_path}{filename}.txt", mode="w") as f:
            f.write("{\"data\":[")

        for num, D in enumerate(dates):
            soup = self.retry(D)
            try:
                table = soup.select('table[bgcolor="#FFFFFF"]')[1]
                table_head = table.select('[align="center"]')
                table_currency = table.select('[align="left"]')
                table_exchange = table.select('[align="right"]')

                table_head = list(map(lambda x : x.text,table_head))
                table_currency = list(map(lambda x : x.text,table_currency))
                table_exchange = list(map(lambda x : x.text,table_exchange))

                table_buy = []
                table_sale = []

                with open(f"{txt_path}{filename}.txt", mode="a") as f:
                    f.write("{\"date\":")
                    f.write(f'"{D}"')
                    f.write(",\"detail\":")

                for i, j in enumerate(table_exchange):
                    if i%2 == 0:
                        table_buy.append(j)
                    else:
                        table_sale.append(j)
                Json = list(map(lambda x,y,z : {
                    "幣別":x,
                    "買入匯率":y,
                    "賣出匯率":z
                },table_currency,table_buy,table_sale))
                Json = [json.load(json.dumps(i,ensure_ascii=False) for i in Json)]
                with open(f"{txt_path}{filename}.txt", mode="a") as f:
                    f.write(str(Json).replace("\'",""))
                    f.write("}")
                    if num+1 == len(dates):
                      pass
                    else:
                      f.write(",")

                # data_for_df = {"date":D, "detail":Json}
                print(f"{D} success")

            except:
                print(f"{D} fail")
                pass

        with open(f"{txt_path}{filename}.txt", mode="a") as f:
            f.write("]}")
        return (filename, df)

    def store(self, filename, df):
        df.to_csv(f"{csv_path}{filename}.csv", encoding="utf-8")
        df.to_csv(f"{newest_path}{filename}.csv", encoding="utf-8")
# if __name__ == "__main__":
c = exchange_rate()




2022/10/01 fail
2022/10/02 fail
2022/10/03 fail
2022/10/04 fail
2022/10/05 fail
2022/10/06 fail
2022/10/07 fail
2022/10/08 fail
2022/10/09 fail
2022/10/10 fail
2022/10/11 fail
2022/10/12 fail
2022/10/13 fail
2022/10/14 fail
2022/10/15 fail
2022/10/16 fail
2022/10/17 fail
2022/10/18 fail
2022/10/19 fail
2022/10/20 fail
2022/10/21 fail
2022/10/22 fail
2022/10/23 fail
2022/10/24 fail
2022/10/25 fail
2022/10/26 fail
2022/10/27 fail
2022/10/28 fail
2022/10/29 fail
2022/10/30 fail
2022/10/31 fail


費用率與報酬率

In [ ]:
import time
import json 
import math
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
import random
import os
import datetime
import warnings

# 建立資料夾
newest_path = './最新檔/'
log_path = './近五年費用率與報酬率記錄檔/'
dir = ["csv/", "txt/", "err/"]

for i in dir:
  if not os.path.isdir(log_path+i):
    os.makedirs(log_path+i)

if not os.path.isdir(newest_path):
    os.mkdir(newest_path)

# 路徑設定
config_path = "/content/drive/MyDrive/config.json"
csv_path = log_path+dir[0]
txt_path = log_path+dir[1]
err_path = log_path+dir[2]
basicInfo_path = f"{newest_path}境外基金基本資訊.csv"

# 參數設定
delay = random.uniform(1, 5)
warnings.filterwarnings("ignore")
useragent = UserAgent()
now = datetime.datetime.now().strftime("%Y%m%d")    
filename = f'近五年費用率與報酬率'
# basicInfo_path = '/content/drive/MyDrive/'
# config_path = './config.json'
# output_path = './'

c = pd.read_csv(f'{basicInfo_path}')

header = {
    'Host' : 'announce.fundclear.com.tw',
    'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    "Referer": "https://announce.fundclear.com.tw",   
    "user-agent" : useragent.random
}

class expNre(object):
    def __init__(self):
        a = self.crawler()
        a = self.store(a)

    def get_source(self, url):
        times = 0
        while times<3 :
            try:
                res=requests.get(url,headers = header)
                soup = BeautifulSoup(res.text, 'html.parser')
                soup.select("td.TableHead")[1]#確認有值
                return soup
            except:
                times+=1
                print(f"try {times} time, {url}")
                pass

    def crawler(self):
        J = []
        with open(f'{txt_path}{filename}{now}.txt', mode = "w") as f:
            f.write("{data:[")
        for j,i in enumerate(c['基金代碼']):
            url = f"https://announce.fundclear.com.tw/MOPSFundWeb/A01_12.jsp?fundId={i}"

            soup = self.get_source(url)

            TableHead = list(map(lambda x: x.text, soup.select("td.TableHead")))
            rate = list(map(lambda x: x.text, soup.select('td[align="right"]')))
            year = list(map(lambda x: x.text, soup.select('td[width="17%"]')))
            exp = rate[:5]
            re = rate[5:]
            Json = list(map(lambda x,y,z:{
                '基金代碼':i,
                TableHead[0]:x,
                TableHead[1]:y,
                TableHead[2]:z
            }, year,exp,re))
            for x in Json:
                with open(f'{txt_path}{filename}{now}.txt', mode = "a") as f:
                    f.write(str(x))
                    f.write(',')

            J = J+Json
            print(f"{j}/{len(c)}")
            time.sleep(delay)  
            if j == 10:
              break  

        with open(f'{txt_path}{filename}{now}.txt', mode = "a") as f:
            f.write("]}")
        return J

    def store(self, J):
        df = pd.DataFrame(data = J) 
        df.to_csv(f"{csv_path}{filename}{now}.csv", encoding ="utf-8")
        df.to_csv(f"{newest_path}{filename}.csv", encoding ="utf-8")

if __name__ == "__main__":
    E = expNre()


0/11
1/11
2/11
3/11
4/11
5/11
6/11
7/11
8/11
9/11
10/11
